### V Ivanouskaya
16.03.2024
homework TF
v1

Решить задачу просрочки кредита используя для прогназирования нейронные сети. Посмотреть работу с несбалансированными данными. 
Набор данных представляет собой исторические данные по 251503 заемщикам (https://www.kaggle.com/c/GiveMeSomeCredit/data ).
Задача является бинарной классификацией. 
Цель - предсказать будет ли тот или иной заемщик испытывать финансовые трудности в ближайшие 2 года, т.е. будет ли просрочка по займу. 
Выборка разделена на тренировочную и тестовую ( 150000 в тренировочной части, 101503 в тестовой).

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [48]:
train_data = pd.read_csv('cs-training.csv')
test_data = pd.read_csv('cs-test.csv')

### Подготовка данных

In [49]:
train_data.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


###### RevolvingUtilizationOfUnsecuredLines: Процент использования кредитных линий.
######  age: Возраст заемщика.
######  NumberOfTime30-59DaysPastDueNotWorse: Количество раз, когда заемщик опаздывал с платежом на 30-59 дней.
######  DebtRatio: Долговая нагрузка.
######  MonthlyIncome: Ежемесячный доход.
###### NumberOfOpenCreditLinesAndLoans: Количество открытых кредитных линий и займов.
###### NumberOfTimes90DaysLate: Количество раз, когда заемщик опаздывал с платежом более чем на 90 дней.
###### NumberRealEstateLoansOrLines: Количество ипотечных кредитов и кредитных линий.
###### NumberOfTime60-89DaysPastDueNotWorse: Количество раз, когда заемщик опаздывал с платежом на 60-89 дней.
###### NumberOfDependents: Количество иждивенцев.

In [50]:
# Удаление лишнего столбца
train_data.drop(columns=['Unnamed: 0'], inplace=True)

In [51]:
train_data.isnull().sum()

SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

In [52]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7   NumberOfTimes90DaysLate               150000 non-null  int64  
 8   NumberRealEstateLoansOrLines          150000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 10  NumberOfDependents                    146076 non-null  float64
dtype

In [53]:
# Обработка пропущенных значений
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
train_data[['MonthlyIncome', 'NumberOfDependents']] = imputer.fit_transform(train_data[['MonthlyIncome', 'NumberOfDependents']])

In [54]:
# Нормализация данных, кроме целевой переменной
scaler = StandardScaler()
X = train_data.drop(columns=['SeriousDlqin2yrs'])
y = train_data['SeriousDlqin2yrs']
X_scaled = scaler.fit_transform(X)

In [55]:
# Проверка баланса классов
class_balance = y.value_counts(normalize=True)
class_balance

SeriousDlqin2yrs
0    0.93316
1    0.06684
Name: proportion, dtype: float64

0    1
1    0
2    0
3    0
4    0
Name: SeriousDlqin2yrs, dtype: int64

In [57]:
# Разделение данных на тренировочную и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
# Проверка размеров полученных выборок
X_train.shape, X_val.shape, y_train.shape, y_val.shape


((120000, 10), (30000, 10), (120000,), (30000,))

In [60]:
# Вычисление весов классов для учета несбалансированности
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

In [61]:
# Вычисление весов классов для учета несбалансированности
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

In [63]:
# Построение модели нейронной сети
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [64]:
# Компиляция модели
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [65]:
# Обучение модели
history = model.fit(X_train, y_train, 
                    epochs=10, 
                    batch_size=64, 
                    validation_data=(X_val, y_val), 
                    class_weight=class_weight_dict,
                    verbose=1)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7429 - loss: 0.5643 - val_accuracy: 0.8421 - val_loss: 0.4652
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8087 - loss: 0.5190 - val_accuracy: 0.7952 - val_loss: 0.5303
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8140 - loss: 0.5123 - val_accuracy: 0.8322 - val_loss: 0.4559
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8146 - loss: 0.5083 - val_accuracy: 0.8247 - val_loss: 0.4721
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8148 - loss: 0.5064 - val_accuracy: 0.8002 - val_loss: 0.5096
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8179 - loss: 0.4983 - val_accuracy: 0.8078 - val_loss: 0.4970
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8169 - loss: 0.4921 - val_accuracy: 0.7992 - val_loss: 0.5278
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8187 - loss: 0.4934 - 

#### Результ обучения:

###### точность на обучающем наборе данных : модель достигает точности около 81.6% на последней эпохе.
###### точность на валидационном наборе данных : модель показывает валидационную точность около 82.2% на последней эпохе.
###### функция потерь: значение функции потерь снижается как на обучающем, так и на валидационном наборе данных с каждой эпохой и разница между обучающим и валидационным наборами не очень велика, что хорошо с точки зрения предотвращения переобучения.

##### Вычисление дополнительных метрик на валидационном и тестовом наборах данных.

In [68]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve
# Предсказание вероятностей для валидационного набора данных
y_val_pred_probs = model.predict(X_val)
y_val_pred = (y_val_pred_probs > 0.5).astype("int32")
y_val_pred

938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [69]:
# Вычисление метрик
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_pred_probs)
print('Precision:', precision)
print('Recall:', recall)
print('F1 score:', f1)
print('ROC AUC:', roc_auc)

Precision: 0.2281449893390192
Recall: 0.6937655860349127
F1 score: 0.3433720069118736
ROC AUC: 0.8346352016012835


##### Выводы:
###### Precision : значение точности  низкое, из всех предсказанных моделью как положительных случаев просрочки кредита только около 22.8% действительно являются таковыми.
###### Recall : модель смогла корректно идентифицировать примерно 69.4% всех реальных случаев просрочки.
###### F1 Score: F1-мера составляет около 0.34, что ниже, чем может быть. Это говорит о том, что между точностью и полнотой есть значительный дисбаланс.
###### ROC AUC: площадь под кривой ROC (AUC) высока и составляет 0.84, что  считается хорошим результатом. Это показывает, модель обладает хорошей способностью различать классы.